## 1. Librerías

In [1]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [2]:
# Exploración inicial de datos
def check_df(df, tipo):
  # tipo == 'simple' - Solo muestra volumenes y cabecera
  if tipo == 'simple':
    print("¿Cuántas filas y columnas hay en el conjunto de datos?")
    num_filas, num_columnas = df.shape
    print("\tHay {:,} filas y {:,} columnas.".format(num_filas, num_columnas))
    print('\n########################################################################################')

    print("¿Cuáles son las primeras dos filas del conjunto de datos?")
    display(df.head(2))
    print('\n########################################################################################')

    print("¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?")
    display(df.sample(2))
    print('\n########################################################################################')

  else:

    print("¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?")
    display(df.sample(2))
    print('\n########################################################################################')
    
    print("¿Cuántas filas y columnas hay en el conjunto de datos?")
    num_filas, num_columnas = df.shape
    print("\tHay {:,} filas y {:,} columnas.".format(num_filas, num_columnas))
    print('\n########################################################################################')

    print("¿Cuáles son las columnas del conjunto de datos?")
    for i in list(df.columns):
      print('\t - ' + i)
    print('\n########################################################################################')

    print("¿Cuál es el tipo de datos de cada columna?")
    print(df.dtypes)
    print('\n########################################################################################')

    print("¿Cuántas columnas hay de cada tipo de datos?")
    print(df.dtypes.value_counts())
    print('\n########################################################################################')

    print("¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFrame?")
    print(df.info())
    print('\n########################################################################################')

    print("¿Cuántos valores únicos tiene cada columna?")
    print(df.nunique())
    print('\n########################################################################################')

    print("¿Cuáles son las estadísticas descriptivas básicas de las columnas numéricas?")
    display(df.describe(include = 'all').fillna(''))
    print('\n########################################################################################')

    print("¿Hay valores nulos en el conjunto de datos?")
    print(df.isnull().sum().sort_values(ascending = False))
    print('\n########################################################################################')

    print("¿Cuál es el porcentaje de valores nulos en cada columna?")
    print(round((df.isnull().sum()/len(df)*100), 2).sort_values(ascending = False))
    print('\n########################################################################################')

    print("¿Cuantos países tenemos en el dataset?")
    if 'Code' in df.columns:
      num_paises = df['Code'].nunique()
    elif 'Country' in df.columns:
      num_paises = df['Country'].nunique()
    print(num_paises)
    print('\n########################################################################################')

    print("¿Qué rango y cantidad de años tenemos en el dataset?")
    print("Nº de años totales:", df['Year'].nunique())
    print(f"Rango de años: {df['Year'].min()} - {df['Year'].max()}")
    print('\n########################################################################################')

In [3]:
# Códigos de países faltantes
def missing_codes(df):
    return df[df['Code'].isna()]['Entity'].unique()

# 2. Data

In [70]:
data_folder = '/Users/Alis/Desktop/MASTER NUCLIO/TRABAJO FINAL/Exploracion_Inicial/resultados_normalizados/'
data_folder_2 = 'data/'

annual_number_of_deaths_by_cause_path = data_folder + 'df_deaths_100k.csv'
death_rate_from_suicides_gho_path = data_folder + 'df_suicides.csv'
gdp_per_capita_penn_world_table_path = data_folder + 'df_gdp.csv'
global_vaccination_coverage_path = data_folder + 'df_vaccination.csv'
homicide_rate_unodc_path = data_folder + 'df_homicide.csv'
life_expectancy_path = data_folder + 'df_life.csv'
poverty_explorer_path = data_folder + 'df_poverty.csv'
public_healthcare_spending_share_gdp_path = data_folder + 'df_spending.csv'
population_demography = data_folder + 'df_poblation.csv'


In [71]:
df_deaths = pd.read_csv(annual_number_of_deaths_by_cause_path)
df_suicides = pd.read_csv(death_rate_from_suicides_gho_path)
df_gdp = pd.read_csv(gdp_per_capita_penn_world_table_path)
df_vaccination = pd.read_csv(global_vaccination_coverage_path)
df_homicide = pd.read_csv(homicide_rate_unodc_path)
df_life = pd.read_csv(life_expectancy_path)
df_poverty = pd.read_csv(poverty_explorer_path)
df_spending = pd.read_csv(public_healthcare_spending_share_gdp_path)
df_population = pd.read_csv(population_demography)

# 3. Normalización de los datos

En este apartado vamos a filtrar los datasets de varias maneras:

- Por países: usaremos sólo los países que son comunes en todos los datasets, teniendo en cuenta que hay algunos que aparecen con nombres distintos. Por lo tanto, se eliminarán las agrupaciones de países y aquellos que no son países como tal, sino que son colonias o territorios dependientes de otros países.
- Por años: el marco temporal a trabajar será 2000-2019, por lo que filtraremos por estas fechas.
- Eliminación de columnas que no vayamos a usar: en los datasets de referentes a las vacunaciones y la pobreza hay muchas variables que no se usarán para el estudio, por lo que vamos a proceder a quitarlas para simplificar posteriormente la unión de los datos.
- Unión de los datasets en uno único.

## 3.1 Países

Cambiamos la columna "Country" por "Entity" del dataset de pobreza para luego usar esta columna común para identificar los países comunes

In [72]:
df_poverty.head()

,Country,Year,Share below $1 a day,Share below $2.15 a day,Share below $3.65 a day,Share below $6.85 a day,Share below $10 a day,Share below $20 a day,Share below $30 a day,Share below $40 a day,...,decile10_avg,bottom50_share,middle40_share,palma_ratio,s80_s20_ratio,p90_p10_ratio,p90_p50_ratio,p50_p10_ratio,Entity,ISO3
0,Albania,1996,0.057416,0.534846,8.898443,47.879143,75.623880,98.702095,99.89742,99.951650,...,16.830648,31.233019,47.551407,0.928335,3.945872,3.566505,1.888392,1.888647,Albania,ALB
1,Albania,2002,0.046371,1.092647,11.388324,52.090965,76.705050,96.528860,98.94813,99.762024,...,20.623796,28.586811,45.977550,1.215056,4.831625,3.995204,2.092198,1.909572,Albania,ALB
2,Albania,2005,0.000000,0.591086,7.289305,40.428430,68.453090,95.405670,98.85736,99.546940,...,22.311052,29.289268,46.369564,1.142718,4.662236,3.874336,1.979621,1.957110,Albania,ALB
3,Albania,2008,0.000000,0.199925,3.902613,33.431850,62.768870,94.582830,98.50923,99.344450,...,24.551666,29.854542,45.687150,1.114657,4.395911,3.574216,1.957677,1.825743,Albania,ALB
4,Albania,2012,0.024101,0.620691,4.805442,36.667750,65.062386,95.520160,99.07353,99.742485,...,21.827140,30.110823,46.954838,1.041193,4.272574,3.630154,1.942752,1.868563,Albania,ALB


In [73]:
df_poverty.drop(columns=['Country'], inplace=True)

In [74]:
# Listado de datasets
dataset_list = [df_deaths,df_suicides,df_gdp,df_vaccination,df_homicide,df_life,df_poverty,df_spending,df_population]

# Cambiamos la columna para el dataset de población (externo)
df_population.rename(columns = {"Country name": "Entity"},inplace = True)

In [75]:
# Extraer la columna 'Entity' de cada DataFrame y almacenarla en una lista de conjuntos
entities_list = [set(df['Entity']) for df in dataset_list]

# Encontrar la intersección de todos los conjuntos en entities_list
common_entities = set.intersection(*entities_list)

print("Valores comunes en todos los DataFrames:")
for enttity in common_entities:
    print(enttity) # Celda ocultada a través de VSC


Valores comunes en todos los DataFrames:
Lebanon
Georgia
Uzbekistan
Philippines
Hungary
Rwanda
Ecuador
Niger
Bhutan
Serbia
Panama
Peru
United Kingdom
Botswana
Estonia
Haiti
Costa Rica
Japan
Belarus
Türkiye
Sierra Leone
Latvia
Luxembourg
Israel
Slovenia
France
Armenia
Italy
Netherlands
Zambia
Iceland
Namibia
Uganda
Trinidad and Tobago
Denmark
Switzerland
Mongolia
India
Viet Nam
Egypt
Morocco
Finland
South Africa
Sri Lanka
Colombia
Poland
Bulgaria
Malawi
Russian Federation
Lithuania
Honduras
Belize
Albania
Ukraine
Austria
Nepal
Burundi
Fiji
Kenya
Turkmenistan
Cyprus
Australia
Ghana
Guinea-Bissau
Nicaragua
Portugal
Angola
Myanmar
Canada
Zimbabwe
Brazil
Iraq
Malaysia
Tanzania, United Republic of
Czechia
United States
Liberia
Germany
Ethiopia
Pakistan
Bolivia, Plurinational State of
Mauritania
Algeria
Nigeria
Mozambique
Iran, Islamic Republic of
Mexico
Tunisia
United Arab Emirates
Azerbaijan
Norway
Jamaica
Yemen
Romania
Jordan
Belgium
North Macedonia
Indonesia
Uruguay
Paraguay
Korea, Republ

In [76]:
print("Nº de países en común:",len(common_entities))

Nº de países en común: 122


In [77]:
import pycountry
# Función para convertir nombres de países a códigos ISO 3166-1 alpha-3 (alpha-3 significa 3 digitos, ESP, FRA,DEU ...)
def country_to_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Convertir nombres de países a códigos ISO 3166-1 alpha-3
for df in dataset_list:
    df['ISO3'] = df['Entity'].apply(country_to_iso3)

# Extraer la columna 'ISO3' de cada DataFrame y almacenarla en una lista de conjuntos
iso3_list = [set(df['ISO3'].dropna()) for df in dataset_list]

# Encontrar la intersección de todos los conjuntos en iso3_list
common_iso3 = set.intersection(*iso3_list)
print("Etiquetas:",common_iso3)
# Convertir los códigos ISO 3166-1 alpha-3 de vuelta a nombres de países
common_countries = [pycountry.countries.get(alpha_3=iso3).name for iso3 in common_iso3]

print("Valores comunes en todos los DataFrames (usando códigos ISO 3166-1 alpha-3):")
for country in common_countries:
    print(country)


Etiquetas: {'BGR', 'MNG', 'DEU', 'MMR', 'HRV', 'BRA', 'TUN', 'MWI', 'CYP', 'NPL', 'GUY', 'IRL', 'MKD', 'VEN', 'MYS', 'FRA', 'COL', 'BGD', 'CRI', 'LKA', 'MDA', 'SRB', 'BWA', 'GTM', 'PER', 'BEL', 'VNM', 'KAZ', 'ITA', 'BDI', 'IDN', 'TJK', 'CZE', 'PRT', 'TZA', 'NLD', 'EGY', 'THA', 'PRY', 'NAM', 'LSO', 'JOR', 'SLV', 'HND', 'MAR', 'UZB', 'YEM', 'LTU', 'USA', 'MRT', 'GEO', 'EST', 'AUT', 'FJI', 'SVN', 'ECU', 'IND', 'AZE', 'SVK', 'CHL', 'MOZ', 'BOL', 'ZWE', 'MEX', 'ROU', 'IRN', 'LBR', 'PAN', 'ALB', 'AGO', 'TKM', 'ISL', 'PHL', 'ARE', 'TUR', 'LBN', 'BLR', 'RUS', 'TTO', 'SLE', 'GNB', 'ISR', 'ESP', 'NOR', 'ARM', 'POL', 'LVA', 'NER', 'URY', 'GBR', 'GHA', 'PAK', 'NIC', 'SYR', 'JPN', 'KEN', 'ETH', 'NGA', 'KOR', 'DZA', 'UKR', 'BTN', 'CAN', 'HTI', 'GRC', 'MNE', 'ZMB', 'JAM', 'CMR', 'IRQ', 'LUX', 'BLZ', 'ZAF', 'DNK', 'SWE', 'FIN', 'CHN', 'CHE', 'HUN', 'AUS', 'UGA', 'RWA'}
Valores comunes en todos los DataFrames (usando códigos ISO 3166-1 alpha-3):
Bulgaria
Mongolia
Germany
Myanmar
Croatia
Brazil
Tunisia


In [78]:
print("Nº de países en común:",len(common_countries))

Nº de países en común: 122


In [79]:
# Lista de datasets
datasets = [df_deaths, df_suicides, df_gdp, df_vaccination, df_homicide, df_life, df_poverty, df_spending, df_population]

# Función para filtrar dataset por los países comunes
def filter_dataset_by_countries(df, country_column, common_countries):
    return df[df[country_column].isin(common_countries)]

country_column = 'Entity'

# Mostrar el shape antes y después de la actualización
for i, df in enumerate(datasets):
    print(f"Shape del dataset {i+1} antes de la actualización: {df.shape}")

# Aplicar el filtro a todos los datasets
filtered_datasets = [filter_dataset_by_countries(df, country_column, common_countries) for df in datasets] 

# Mostrar el shape antes y después de la actualización
for i, df in enumerate(filtered_datasets):
    print(f"Shape del dataset {i+1} después de la actualización: {df.shape}")

# Desempaquetar los datasets filtrados
df_deaths, df_suicides, df_gdp, df_vaccination, df_homicide, df_life, df_poverty, df_spending, df_population = filtered_datasets 

# Mostrar el resultado del primer dataset filtrado como ejemplo
print("Dataset df_deaths filtrado:")
print(df_deaths)

Shape del dataset 1 antes de la actualización: (6150, 39)
Shape del dataset 2 antes de la actualización: (3880, 5)
Shape del dataset 3 antes de la actualización: (10108, 5)
Shape del dataset 4 antes de la actualización: (7897, 15)
Shape del dataset 5 antes de la actualización: (4204, 5)
Shape del dataset 6 antes de la actualización: (10722, 5)
Shape del dataset 7 antes de la actualización: (2602, 110)
Shape del dataset 8 antes de la actualización: (4014, 5)
Shape del dataset 9 antes de la actualización: (18288, 42)
Shape del dataset 1 después de la actualización: (3660, 39)
Shape del dataset 2 después de la actualización: (2440, 5)
Shape del dataset 3 después de la actualización: (6969, 5)
Shape del dataset 4 después de la actualización: (4744, 15)
Shape del dataset 5 después de la actualización: (2741, 5)
Shape del dataset 6 después de la actualización: (10722, 5)
Shape del dataset 7 después de la actualización: (1892, 110)
Shape del dataset 8 después de la actualización: (2404, 5)
Sh

In [80]:
df_deaths['Entity'].nunique()

122

In [81]:
def unique(filtered_datasets):
    for i, df in enumerate(filtered_datasets):
        num_unique = df['Entity'].nunique()
        print(f"Dataset {i+1}: Número de países únicos: {num_unique}")

In [82]:
#Aplicamos la función al listado de datasets filtrados, para ver que hemos realizado bien el filtro
unique(filtered_datasets)

Dataset 1: Número de países únicos: 122
Dataset 2: Número de países únicos: 122
Dataset 3: Número de países únicos: 122
Dataset 4: Número de países únicos: 122
Dataset 5: Número de países únicos: 122
Dataset 6: Número de países únicos: 122
Dataset 7: Número de países únicos: 122
Dataset 8: Número de países únicos: 122
Dataset 9: Número de países únicos: 122


Creamos un diccionario con los datasets para asegurarnos de que los filtros se aplican correctamente a todos ellos y se mantienen

In [83]:
dataset_names = ['df_deaths', 'df_suicides', 'df_gdp', 'df_vaccination', 'df_homicide', 'df_life', 'df_poverty', 'df_spending', 'df_population']
# Crear un diccionario para almacenar los DataFrames filtrados
filtered_datasets_dict = {}

# Almacenar los DataFrames filtrados en el diccionario
for name, df in zip(dataset_names, filtered_datasets):
    filtered_datasets_dict[name + '_filtered'] = df.copy()

# Verificar los DataFrames almacenados en el diccionario
for name, df in filtered_datasets_dict.items():
    print(f"Shape del dataset {name}: {df.shape}")
    print(f"Primeras filas del dataset {name}:")
    print(df.head())

Shape del dataset df_deaths_filtered: (3660, 39)
Primeras filas del dataset df_deaths_filtered:
     Entity Code  Year ISO3_x  Population ISO3_y  Country  \
30  Albania  ALB  1990    ALB     3295073    ALB  Albania   
31  Albania  ALB  1991    ALB     3302087    ALB  Albania   
32  Albania  ALB  1992    ALB     3303738    ALB  Albania   
33  Albania  ALB  1993    ALB     3300715    ALB  Albania   
34  Albania  ALB  1994    ALB     3294001    ALB  Albania   

    ('Deaths - Meningitis - Sex: Both - Age: All Ages (Number)_per_100k',)  \
30                                           3.247273                        
31                                           3.391794                        
32                                           3.269024                        
33                                           3.090240                        
34                                           2.823314                        

    ("Deaths - Alzheimer's disease and other dementias - Sex: Both -

Como vemos, hemos reducido el nombre de países únicos a 122 en todos los datasets. ya que hemos eliminado aquellos que no eran comunes en todos y las agrupaciones de países

## 3.2 Años de estudio: 2000-2019

In [84]:
def filter_by_year(df, year_column='Year', start_year=2000, end_year=2019):
    df[year_column] = pd.to_numeric(df[year_column], errors='coerce')
    return df[(df[year_column] >= start_year) & (df[year_column] <= end_year)]

# Aplicar el filtro de años a los DataFrames filtrados y actualizarlos en el diccionario
for name, df in filtered_datasets_dict.items():
    filtered_datasets_dict[name] = filter_by_year(df)

# Verificar los DataFrames después de aplicar el filtro de años
for name, df in filtered_datasets_dict.items():
    print(f"Shape del dataset {name} después del filtro de años: {df.shape}")
    print(f"Primeras filas del dataset {name} después del filtro de años:")
    print(df.head())


Shape del dataset df_deaths_filtered después del filtro de años: (2440, 39)
Primeras filas del dataset df_deaths_filtered después del filtro de años:
     Entity Code  Year ISO3_x  Population ISO3_y  Country  \
40  Albania  ALB  2000    ALB     3182027    ALB  Albania   
41  Albania  ALB  2001    ALB     3153615    ALB  Albania   
42  Albania  ALB  2002    ALB     3123554    ALB  Albania   
43  Albania  ALB  2003    ALB     3092993    ALB  Albania   
44  Albania  ALB  2004    ALB     3062629    ALB  Albania   

    ('Deaths - Meningitis - Sex: Both - Age: All Ages (Number)_per_100k',)  \
40                                           1.508472                        
41                                           1.331805                        
42                                           1.248578                        
43                                           1.163921                        
44                                           1.077506                        

    ("Deaths -

## 2.3 Eliminación columnas ISO

In [85]:
# Función para eliminar columnas con nombres específicos
def remove_iso_columns(df):
    return df.drop(columns=[col for col in df.columns if 'ISO3' in col], errors='ignore')

# Aplicar la eliminación de columnas ISO3 a los DataFrames filtrados y actualizarlos en el diccionario
for name, df in filtered_datasets_dict.items():
    filtered_datasets_dict[name] = remove_iso_columns(df)

# Verificar los DataFrames después de eliminar las columnas
for name, df in filtered_datasets_dict.items():
    print(f"Shape del dataset {name} después de eliminar columnas ISO3: {df.shape}")
    print(f"Columnas del dataset {name} después de eliminar columnas ISO3:")
    print(df.columns)

Shape del dataset df_deaths_filtered después de eliminar columnas ISO3: (2440, 36)
Columnas del dataset df_deaths_filtered después de eliminar columnas ISO3:
Index(['Entity', 'Code', 'Year', 'Population', 'Country',
       '('Deaths - Meningitis - Sex: Both - Age: All Ages (Number)_per_100k',)',
       '("Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number)_per_100k",)',
       '("Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number)_per_100k",)',
       '('Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number)_per_100k',)',
       '('Deaths - Malaria - Sex: Both - Age: All Ages (Number)_per_100k',)',
       '('Deaths - Drowning - Sex: Both - Age: All Ages (Number)_per_100k',)',
       '('Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Number)_per_100k',)',
       '('Deaths - Maternal disorders - Sex: Both - Age: All Ages (Number)_per_100k',)',
       '('Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number)_per_

Ahora que ya tenemos los datasets filtrados por países, marco temporal y sin las columnas de ISO, los sacamos del diccionario

In [86]:
# Desempaquetar los DataFrames filtrados del diccionario y asignarlos a variables específicas
df_deaths_filtered = filtered_datasets_dict['df_deaths_filtered']
df_suicides_filtered = filtered_datasets_dict['df_suicides_filtered']
df_gdp_filtered = filtered_datasets_dict['df_gdp_filtered']
df_vaccination_filtered = filtered_datasets_dict['df_vaccination_filtered']
df_homicide_filtered = filtered_datasets_dict['df_homicide_filtered']
df_life_filtered = filtered_datasets_dict['df_life_filtered']
df_poverty_filtered = filtered_datasets_dict['df_poverty_filtered']
df_spending_filtered = filtered_datasets_dict['df_spending_filtered']
df_population_filtered = filtered_datasets_dict['df_population_filtered']

# Verificar los DataFrames desempacados
print("Número de países únicos en df_deaths_filtered después del filtrado:", df_deaths_filtered['Entity'].nunique())

Número de países únicos en df_deaths_filtered después del filtrado: 122


## 2.4 Revisión de columnas por cada dataset

### 2.4.1 Causas de muerte

Este dataset tiene nombres de columnas muy largos, por lo que vamos a proceder a cambiarlos para facilitar el análisis posterior.
Usaremos un bucle basado en expresiones regulares que identifican el nombre de la causa de muerte (que se encuentra entre guiones)

In [87]:
import re

def extract_keyword(column_name):
    # Utilizar una expresión regular para extraer la palabra clave entre los guiones
    match = re.search(r'-\s*(.*?)\s*-', column_name)
    if match:
        return match.group(1).strip()
    else:
        return column_name

def rename_columns(df):
    new_columns = {}
    for col in df.columns:
        new_columns[col] = extract_keyword(col)
    return df.rename(columns=new_columns)

# Aplicar el renombramiento al dataset df_deaths_filtered
df_deaths_filtered = rename_columns(df_deaths_filtered)

# Verificar las columnas renombradas
print("Columnas renombradas de df_deaths_filtered:")
print(df_deaths_filtered.columns)

Columnas renombradas de df_deaths_filtered:
Index(['Entity', 'Code', 'Year', 'Population', 'Country', 'Meningitis',
       'Alzheimer's disease and other dementias', 'Parkinson's disease',
       'Nutritional deficiencies', 'Malaria', 'Drowning',
       'Interpersonal violence', 'Maternal disorders', 'HIV/AIDS',
       'Drug use disorders', 'Tuberculosis', 'Cardiovascular diseases',
       'Lower respiratory infections', 'Neonatal disorders',
       'Alcohol use disorders', 'Self', 'Exposure to forces of nature',
       'Diarrheal diseases', 'Environmental heat and cold exposure',
       'Neoplasms', 'Conflict and terrorism', 'Diabetes mellitus',
       'Chronic kidney disease', 'Poisonings', 'Protein', 'Road injuries',
       'Chronic respiratory diseases',
       'Cirrhosis and other chronic liver diseases', 'Digestive diseases',
       'Fire, heat, and hot substances', 'Acute hepatitis'],
      dtype='object')


In [88]:
df_deaths_filtered.drop(columns='Country', inplace=True)


In [89]:
df_deaths_filtered.reset_index(drop=True)

,Entity,Code,Year,Population,Meningitis,Alzheimer's disease and other dementias,Parkinson's disease,Nutritional deficiencies,Malaria,Drowning,...,Diabetes mellitus,Chronic kidney disease,Poisonings,Protein,Road injuries,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,"Fire, heat, and hot substances",Acute hepatitis
0,Albania,ALB,2000,3182027,1.508472,13.764811,3.676902,0.691383,0.000000,2.859812,...,4.431138,7.668068,0.754236,0.659957,8.139466,23.664161,8.328025,14.173356,0.659957,0.031427
1,Albania,ALB,2001,3153615,1.331805,14.301048,3.741738,0.507354,0.000000,2.536771,...,4.439350,7.515185,0.729322,0.475645,8.149378,22.387007,8.085958,13.603436,0.602483,0.031710
2,Albania,ALB,2002,3123554,1.248578,15.078977,4.097896,0.416193,0.000000,2.433126,...,4.834237,7.779600,0.704326,0.352163,8.579970,22.762533,8.227807,13.670326,0.608281,0.032015
3,Albania,ALB,2003,3092993,1.163921,15.809929,4.558691,0.323311,0.000000,2.263180,...,5.269976,8.276773,0.646623,0.290980,9.279038,23.731059,8.600084,14.161041,0.614292,0.032331
4,Albania,ALB,2004,3062629,1.077506,16.358495,4.799798,0.261213,0.000000,2.089708,...,5.322225,8.358832,0.587730,0.228562,9.468989,23.378607,8.750652,14.301438,0.587730,0.032652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,Zimbabwe,ZWE,2015,14154937,10.166064,5.326763,1.518905,21.328248,17.788846,5.439798,...,22.437401,14.892330,2.691640,21.123372,16.764469,19.434915,13.818500,29.685756,4.464873,1.031442
2436,Zimbabwe,ZWE,2016,14452705,10.081158,5.306965,1.515287,21.144831,14.184196,5.542215,...,22.549412,14.945299,2.719214,20.944176,16.854976,19.290507,13.575313,29.503128,4.483590,1.010192
2437,Zimbabwe,ZWE,2017,14751101,9.897566,5.294520,1.511752,20.269673,14.344692,5.545349,...,22.459340,14.887024,2.698104,20.079857,16.764850,19.103659,13.605764,29.435091,4.433567,0.976198
2438,Zimbabwe,ZWE,2018,15052191,9.633149,5.281623,1.508086,19.385882,13.871735,5.480930,...,22.461846,14.881554,2.657420,19.199863,16.668670,18.927477,13.486409,29.078823,4.364813,0.923454


In [90]:
df_deaths_100k = df_deaths_filtered.copy()

### 2.4.2 Vacunaciones

Este dataset contiene muchos nulos que debemos tratar y columnas que debemos simplificar

In [91]:
check_df(df_vaccination_filtered,'')

¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,Entity,Code,Year,BCG (% of one-year-olds immunized),HepB3 (% of one-year-olds immunized),Hib3 (% of one-year-olds immunized),IPV1 (% of one-year-olds immunized),MCV1 (% of one-year-olds immunized),PCV3 (% of one-year-olds immunized),Pol3 (% of one-year-olds immunized),RCV1 (% of one-year-olds immunized),RotaC (% of one-year-olds immunized),YFV (% of one-year-olds immunized),DTP3 (% of one-year-olds immunized)
5503,Paraguay,PRY,2008,94.0,87.0,87.0,NaN,91.0,NaN,85.0,91.0,NaN,NaN,89.0
2804,Greece,GRC,2006,NaN,94.0,84.0,NaN,98.0,NaN,98.0,98.0,NaN,NaN,98.0



########################################################################################
¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 2,434 filas y 14 columnas.

########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - Entity
	 - Code
	 - Year
	 - BCG (% of one-year-olds immunized)
	 - HepB3 (% of one-year-olds immunized)
	 - Hib3 (% of one-year-olds immunized)
	 - IPV1 (% of one-year-olds immunized)
	 - MCV1 (% of one-year-olds immunized)
	 - PCV3 (% of one-year-olds immunized)
	 - Pol3 (% of one-year-olds immunized)
	 - RCV1 (% of one-year-olds immunized)
	 - RotaC (% of one-year-olds immunized)
	 - YFV (% of one-year-olds immunized)
	 - DTP3 (% of one-year-olds immunized)

########################################################################################
¿Cuál es el tipo de datos de cada columna?
Entity                                   object
Code                               

,Entity,Code,Year,BCG (% of one-year-olds immunized),HepB3 (% of one-year-olds immunized),Hib3 (% of one-year-olds immunized),IPV1 (% of one-year-olds immunized),MCV1 (% of one-year-olds immunized),PCV3 (% of one-year-olds immunized),Pol3 (% of one-year-olds immunized),RCV1 (% of one-year-olds immunized),RotaC (% of one-year-olds immunized),YFV (% of one-year-olds immunized),DTP3 (% of one-year-olds immunized)
count,2434,2434,2434.0,1964.0,2033.0,1819.0,529.0,2434.0,757.0,2432.0,1765.0,511.0,340.0,2434.0
unique,122,122,,,,,,,,,,,,
top,Albania,ALB,,,,,,,,,,,,
freq,20,20,,,,,,,,,,,,
mean,,,2009.517256,90.956212,86.406296,88.003848,84.194707,88.362777,81.220608,89.015625,91.998867,75.27593,59.202941,89.034922
std,,,5.763448,13.075007,16.371974,15.188492,23.742421,12.34826,21.400136,12.30427,8.949404,22.886681,31.404723,12.521121
min,,,2000.0,0.0,2.0,0.0,0.0,21.0,0.0,8.0,6.0,0.0,0.0,19.0
25%,,,2005.0,90.0,83.0,86.0,81.0,85.0,79.0,85.0,90.0,66.0,36.0,86.0
50%,,,2010.0,95.0,93.0,93.0,96.0,93.0,90.0,94.0,95.0,84.0,71.0,94.0
75%,,,2015.0,98.0,97.0,97.0,98.0,97.0,94.0,97.0,97.0,91.0,85.0,97.0



########################################################################################
¿Hay valores nulos en el conjunto de datos?
YFV (% of one-year-olds immunized)      2094
RotaC (% of one-year-olds immunized)    1923
IPV1 (% of one-year-olds immunized)     1905
PCV3 (% of one-year-olds immunized)     1677
RCV1 (% of one-year-olds immunized)      669
Hib3 (% of one-year-olds immunized)      615
BCG (% of one-year-olds immunized)       470
HepB3 (% of one-year-olds immunized)     401
Pol3 (% of one-year-olds immunized)        2
Entity                                     0
Code                                       0
Year                                       0
MCV1 (% of one-year-olds immunized)        0
DTP3 (% of one-year-olds immunized)        0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
YFV (% of one-year-olds immunized)      86.03
RotaC (% of one-year-olds immu

Según las recomendaciones de la Organización Mundial de la Salud, las vacunas esenciales para garantizar una mayor esperanza de vida al nacer son: 

- TUBERCULOSIS: VACUNA BCG 
- HEPATITIS B: Hepatitis b3
- DTP3 (DIFTERIA/TETANUS/PERTUSSIS)
- POL3 POLIO 

Después de tomar un primer contacto con los datos, vemos que los valores NaN pueden deberse a que los países empezaron a vacunar para determinadas enfermedades en distintos momentos temporales o bien que en muchos de ellos no se vacunan al nacer a los bebés para algunas enfermedades muy específicas. Por todo esto decidimos centrar nuestro análisis en las 5 vacunas principales y de las que tenemos más datos. Las columnas con mayor % de nulos (YFV, RotaC, IPV1, PCV3, RCV1 e Hib3) no se eliminan pero se usarán únicamente en caso de necesitar un análisis más específico sobre alguna de estas enfermedades.

Vacunas en las que nos centraremos: 
- MCV1 (Sarampión)
- DTP3(DIPTHERIA-TETANUS-PERTUSIS)
- pol3 POLIO
- BCG (TUBERCULOSIS)
- HepB3

Las recomendaciones de vacunación son de entre un 90% y 95% para garantizar la inmunización de la población. En base a esta recomendación se realizará el análisis. 

In [92]:
df_vaccination_filtered.isna().sum()

Entity                                     0
Code                                       0
Year                                       0
BCG (% of one-year-olds immunized)       470
HepB3 (% of one-year-olds immunized)     401
Hib3 (% of one-year-olds immunized)      615
IPV1 (% of one-year-olds immunized)     1905
MCV1 (% of one-year-olds immunized)        0
PCV3 (% of one-year-olds immunized)     1677
Pol3 (% of one-year-olds immunized)        2
RCV1 (% of one-year-olds immunized)      669
RotaC (% of one-year-olds immunized)    1923
YFV (% of one-year-olds immunized)      2094
DTP3 (% of one-year-olds immunized)        0
dtype: int64

In [93]:
# Lista de columnas a modificar
columns_to_replace = [
    'Hib3 (% of one-year-olds immunized)',
    'BCG (% of one-year-olds immunized)',
    'HepB3 (% of one-year-olds immunized)',
    'Pol3 (% of one-year-olds immunized)',
    'IPV1 (% of one-year-olds immunized)',
    'PCV3 (% of one-year-olds immunized)',
    'RCV1 (% of one-year-olds immunized)',
    'RotaC (% of one-year-olds immunized)',
    'YFV (% of one-year-olds immunized)',
    'DTP3 (% of one-year-olds immunized)',
    'MCV1 (% of one-year-olds immunized)']



Ahora cambiamos los nombres de las columnas para que sean más concisas y sencillas de interpretar para el análisis. Tenemos en mente que los datos son % de niños menores de 1 año vacunados.

In [94]:
new_names = ['Influenza','Tuberculosis','HepatitisB','Polio','Inactivated_polio','Pneumococcus','Rubella','Rotavirus','Yellow_fever','Diphteria_Tetanus_Pertussis','Measles']

# Renombrar las columnas usando un bucle for y zip: la función zip itera por ambas listas simultáneamente
for old_name, new_name in zip(columns_to_replace, new_names):
    df_vaccination_filtered.rename(columns={old_name: new_name}, inplace=True)



In [95]:
# Verificar los cambios
df_vaccination_filtered.head()

,Entity,Code,Year,Tuberculosis,HepatitisB,Influenza,Inactivated_polio,Measles,Pneumococcus,Polio,Rubella,Rotavirus,Yellow_fever,Diphteria_Tetanus_Pertussis
104,Albania,ALB,2000,93.0,96.0,NaN,NaN,95.0,NaN,97.0,NaN,NaN,NaN,97.0
105,Albania,ALB,2001,93.0,96.0,NaN,NaN,95.0,NaN,97.0,95.0,NaN,NaN,97.0
106,Albania,ALB,2002,94.0,96.0,NaN,NaN,96.0,NaN,98.0,96.0,NaN,NaN,98.0
107,Albania,ALB,2003,95.0,97.0,NaN,NaN,93.0,NaN,97.0,93.0,NaN,NaN,97.0
108,Albania,ALB,2004,97.0,99.0,NaN,NaN,96.0,NaN,98.0,96.0,NaN,NaN,97.0


In [96]:
df_1yold_vaccinated = df_vaccination_filtered.copy()

### 2.4.3 Homicidios

In [97]:
#Cambiaremos el nombre de la columna de homicidos por homicides_rate_100k

df_homicide_filtered.rename(columns={'Homicide rate per 100,000 population - Both sexes - All ages':'homicides_rate_100k'},inplace=True)

In [98]:
df_homicide_100k = df_homicide_filtered.copy()

In [99]:
df_homicide_100k[df_homicide_100k['Entity']=='Cameroon']

,Entity,Code,Year,homicides_rate_100k
742,Cameroon,CMR,2000,3.041428
743,Cameroon,CMR,2001,4.873089
744,Cameroon,CMR,2002,6.478559
745,Cameroon,CMR,2003,6.126819
746,Cameroon,CMR,2008,4.132698
747,Cameroon,CMR,2009,6.609979
748,Cameroon,CMR,2010,5.025647
749,Cameroon,CMR,2011,4.083355
750,Cameroon,CMR,2012,4.183964
751,Cameroon,CMR,2015,3.741421


### 2.4.4 Suicidios

In [100]:
df_suicides_filtered.head()


,Entity,Code,Year,Age-standardized suicide rate - Sex: both sexes
40,Albania,ALB,2000,5.23
41,Albania,ALB,2001,4.74
42,Albania,ALB,2002,4.82
43,Albania,ALB,2003,4.94
44,Albania,ALB,2004,4.86


In [101]:
# Lo mismo para los suicidios:

df_suicides_filtered.rename(columns={'Age-standardized suicide rate - Sex: both sexes':'suicides_rate_100k'},inplace=True)

In [102]:
df_suicides_100k = df_suicides_filtered.copy()

### 2.4.5 Pobreza

In [103]:
#Hemos decidido usar como indicadores de pobreza el umbral de pobreza mundial de 2,15$ por día, el índice de Gini y el índice de palma:
columnas_a_conservar=['Year','Entity','Share below $2.15 a day', 'Number below $2.15 a day', 'palma_ratio', 'gini']

df_poverty_filtered = df_poverty_filtered.filter(items=columnas_a_conservar)
df_poverty_filtered.head()

,Year,Entity,Share below $2.15 a day,Number below $2.15 a day,palma_ratio,gini
1,2002,Albania,1.092647,33337,1.215056,0.317390
2,2005,Albania,0.591086,17800,1.142718,0.305957
3,2008,Albania,0.199925,5892,1.114657,0.299847
4,2012,Albania,0.620691,18003,1.041193,0.289605
5,2014,Albania,1.023101,29558,1.376215,0.345989


In [104]:
df_poverty_short = df_poverty_filtered.copy()

### 2.4.6 Inversión en sanidad pública


In [105]:
# Lo mismo para la inversión en sanidad pública:

df_spending_filtered.rename(columns={'Domestic general government health expenditure (% of GDP)':'health%_GDP'},inplace=True)

In [106]:
df_health_spending = df_spending_filtered.copy()

### 2.4.7 PIB

In [107]:
# Lo mismo para el PIB per capita:

df_gdp_filtered.rename(columns={'GDP per capita (output, multiple price benchmarks)':'GDP_capita'},inplace=True)

In [108]:
df_gdp_capita= df_gdp_filtered.copy()

### 2.4.8 Esperanza de Vida

In [109]:
# Lo mismo para la esperanza de vida:
df_life_filtered.rename(columns={'Period life expectancy at birth - Sex: all - Age: 0':'Life_expectancy'},inplace=True)

In [110]:
df_life_exp= df_life_filtered.copy()

# 3. Creación de CSV

Creamos los CSV definitivos con los que realizaremos el análisis

In [48]:
import os
# Lista de DataFrames y sus nombres correspondientes para los archivos CSV
dataframes = {
   'df_life_exp': df_life_exp,
    'df_gdp_capita': df_gdp_capita,
    'df_health_spending': df_health_spending,
    'df_suicides_100k': df_suicides_100k,
    'df_homicide_100k': df_homicide_100k,
    'df_1yold_vaccinated': df_1yold_vaccinated,
    'df_deaths_100k': df_deaths_100k,
    'df_poverty_short': df_poverty_short
}

# Nombre de la carpeta
directory = 'data_study4'

# Crear la carpeta si no existe
if not os.path.exists(directory):
    os.makedirs(directory)

# Guardar cada DataFrame como un CSV dentro de la carpeta
for name, df in dataframes.items():
    file_path = os.path.join(directory, f'{name}.csv')
    df.to_csv(file_path, index=False)

print("DataFrames guardados exitosamente en la carpeta 'data_study4'.")

DataFrames guardados exitosamente en la carpeta 'data_study4'.


# 4. Unión de datos: dataset final

En este apartado se procederá a unir todos los datasets en uno sólo. Además, se añadirán columnas que indiquen el continente y la región de cada uno de los países. También se modificarán nombres de columnas que siguen siendo demasiado largos para trabajar con ellos.

## 4.1 Cargar archivo geografía xlsx

Cargamos el archivo xlsx donde tenemos los países distribuidos por continentes y regiones

In [111]:
df_geography = pd.read_excel('/Users/Alis/Downloads/Countries_regions_continent.xlsx')

In [112]:
df_geography['Entity'].nunique()

122

## 4.2 Unir 8 datasets de estudio

In [113]:
# Lista de DataFrames
data_4 = [
    df_life_exp,
    df_gdp_capita,
    df_poverty_short,
    df_health_spending,
    df_1yold_vaccinated,
    df_suicides_100k,
    df_homicide_100k,
    df_deaths_100k,
    
]

# Comenzamos con el primer DataFrame
df_analysis = data_4[0]

# Itera sobre los DataFrames restantes y los une secuencialmente: creamos el if de Code ya que esta columna no está en el df de pobreza
for df in data_4[1:]:
    if 'Code' in df.columns:
        df_analysis = df_analysis.merge(df, on=['Entity', 'Code', 'Year'], how='outer')
    else:
        df_analysis = df_analysis.merge(df, on=['Entity', 'Year'], how='outer')

# Verifica y ajusta la columna 'Code' si está duplicada
if 'Code_x' in df_analysis.columns and 'Code_y' in df_analysis.columns:
    df_analysis['Code'] = df_analysis['Code_x'].combine_first(df_analysis['Code_y'])
    df_analysis.drop(columns=['Code_x', 'Code_y'], inplace=True)

# Verifica el DataFrame resultante
print(df_analysis.head())


    Entity Code  Year  Life_expectancy  GDP_capita  Share below $2.15 a day  \
0  Albania  ALB  2000          75.4043   5315.4300                      NaN   
1  Albania  ALB  2001          75.6388   5525.6910                      NaN   
2  Albania  ALB  2002          75.8899   5636.1846                 1.092647   
3  Albania  ALB  2003          76.1421   5861.1265                      NaN   
4  Albania  ALB  2004          76.3761   6038.8965                      NaN   

   Number below $2.15 a day  palma_ratio     gini  health%_GDP  ...  \
0                       NaN          NaN      NaN     2.765830  ...   
1                       NaN          NaN      NaN     2.753276  ...   
2                   33337.0     1.215056  0.31739     2.588669  ...   
3                       NaN          NaN      NaN     2.670353  ...   
4                       NaN          NaN      NaN     2.820827  ...   

   Diabetes mellitus  Chronic kidney disease  Poisonings   Protein  \
0           4.431138        

Perfilamos los nombres de las columnas para que todas tengan el mismo formato

In [114]:
new_column_names2 = {}
for col in df_analysis.columns:
    new_col2 = col.replace(' ', '_')
    new_column_names2[col] = new_col2

# Renombrar las columnas del DataFrame cambiando '' por _
df_analysis.rename(columns=new_column_names2, inplace=True)

Realizamos unos cuantos cambios finales:
- Renombramos las columnas de tuberculosis_x y tuberculosis_y ya que una es de % de vacunación y la otra de número de muertes por cada 100k
- Creamos una lista con los nuevos nombres finales de las columnas, llamada short_columns
- Ordenamos las columnas por preferencias para el análisis
- Recordatorio: las 11 columnas de vacunación representan % de menores de 1 año vacunados para esa enfermedad

In [115]:
df_analysis.rename(columns={'Tuberculosis_x':'Tuberculosis_vac','Tuberculosis_y':'Tuberculosis_100k'}, inplace=True)

In [116]:
df_analysis.head()

,Entity,Code,Year,Life_expectancy,GDP_capita,Share_below_$2.15_a_day,Number_below_$2.15_a_day,palma_ratio,gini,health%_GDP,...,Diabetes_mellitus,Chronic_kidney_disease,Poisonings,Protein,Road_injuries,Chronic_respiratory_diseases,Cirrhosis_and_other_chronic_liver_diseases,Digestive_diseases,"Fire,_heat,_and_hot_substances",Acute_hepatitis
0,Albania,ALB,2000,75.4043,5315.4300,NaN,NaN,NaN,NaN,2.765830,...,4.431138,7.668068,0.754236,0.659957,8.139466,23.664161,8.328025,14.173356,0.659957,0.031427
1,Albania,ALB,2001,75.6388,5525.6910,NaN,NaN,NaN,NaN,2.753276,...,4.439350,7.515185,0.729322,0.475645,8.149378,22.387007,8.085958,13.603436,0.602483,0.031710
2,Albania,ALB,2002,75.8899,5636.1846,1.092647,33337.0,1.215056,0.31739,2.588669,...,4.834237,7.779600,0.704326,0.352163,8.579970,22.762533,8.227807,13.670326,0.608281,0.032015
3,Albania,ALB,2003,76.1421,5861.1265,NaN,NaN,NaN,NaN,2.670353,...,5.269976,8.276773,0.646623,0.290980,9.279038,23.731059,8.600084,14.161041,0.614292,0.032331
4,Albania,ALB,2004,76.3761,6038.8965,NaN,NaN,NaN,NaN,2.820827,...,5.322225,8.358832,0.587730,0.228562,9.468989,23.378607,8.750652,14.301438,0.587730,0.032652


In [117]:
df_analysis['Country']=df_analysis['Entity']

## 4.3 Añadir continente y región

In [118]:
df_merged = df_analysis.merge(df_geography[['Entity', 'Continent', 'Region']], on='Entity', how='left')

# Reorganizar las columnas para que 'Continent' y 'Region' estén al lado de 'Entity' y 'Code'
new_column_order = ['Entity', 'Continent', 'Region', 'Code', 'Year'] + [col for col in df_merged.columns if col not in ['Entity', 'Continent', 'Region', 'Code', 'Year']]

df_analysis_reordered = df_merged[new_column_order]

# Verifica los nuevos nombres de las columnas
print(df_analysis_reordered.columns)

Index(['Entity', 'Continent', 'Region', 'Code', 'Year', 'Life_expectancy',
       'GDP_capita', 'Share_below_$2.15_a_day', 'Number_below_$2.15_a_day',
       'palma_ratio', 'gini', 'health%_GDP', 'Tuberculosis_vac', 'HepatitisB',
       'Influenza', 'Inactivated_polio', 'Measles', 'Pneumococcus', 'Polio',
       'Rubella', 'Rotavirus', 'Yellow_fever', 'Diphteria_Tetanus_Pertussis',
       'suicides_rate_100k', 'homicides_rate_100k', 'Population', 'Meningitis',
       'Alzheimer's_disease_and_other_dementias', 'Parkinson's_disease',
       'Nutritional_deficiencies', 'Malaria', 'Drowning',
       'Interpersonal_violence', 'Maternal_disorders', 'HIV/AIDS',
       'Drug_use_disorders', 'Tuberculosis_100k', 'Cardiovascular_diseases',
       'Lower_respiratory_infections', 'Neonatal_disorders',
       'Alcohol_use_disorders', 'Self', 'Exposure_to_forces_of_nature',
       'Diarrheal_diseases', 'Environmental_heat_and_cold_exposure',
       'Neoplasms', 'Conflict_and_terrorism', 'Diabetes_me

In [119]:
# Verifica las primeras filas del DataFrame reorganizado
df_analysis_reordered.head()

,Entity,Continent,Region,Code,Year,Life_expectancy,GDP_capita,Share_below_$2.15_a_day,Number_below_$2.15_a_day,palma_ratio,...,Chronic_kidney_disease,Poisonings,Protein,Road_injuries,Chronic_respiratory_diseases,Cirrhosis_and_other_chronic_liver_diseases,Digestive_diseases,"Fire,_heat,_and_hot_substances",Acute_hepatitis,Country
0,Albania,Europe,Southern Europe,ALB,2000,75.4043,5315.4300,NaN,NaN,NaN,...,7.668068,0.754236,0.659957,8.139466,23.664161,8.328025,14.173356,0.659957,0.031427,Albania
1,Albania,Europe,Southern Europe,ALB,2001,75.6388,5525.6910,NaN,NaN,NaN,...,7.515185,0.729322,0.475645,8.149378,22.387007,8.085958,13.603436,0.602483,0.031710,Albania
2,Albania,Europe,Southern Europe,ALB,2002,75.8899,5636.1846,1.092647,33337.0,1.215056,...,7.779600,0.704326,0.352163,8.579970,22.762533,8.227807,13.670326,0.608281,0.032015,Albania
3,Albania,Europe,Southern Europe,ALB,2003,76.1421,5861.1265,NaN,NaN,NaN,...,8.276773,0.646623,0.290980,9.279038,23.731059,8.600084,14.161041,0.614292,0.032331,Albania
4,Albania,Europe,Southern Europe,ALB,2004,76.3761,6038.8965,NaN,NaN,NaN,...,8.358832,0.587730,0.228562,9.468989,23.378607,8.750652,14.301438,0.587730,0.032652,Albania


In [120]:
len(df_analysis_reordered.columns)

58

In [121]:
df_analysis_reordered.columns

Index(['Entity', 'Continent', 'Region', 'Code', 'Year', 'Life_expectancy',
       'GDP_capita', 'Share_below_$2.15_a_day', 'Number_below_$2.15_a_day',
       'palma_ratio', 'gini', 'health%_GDP', 'Tuberculosis_vac', 'HepatitisB',
       'Influenza', 'Inactivated_polio', 'Measles', 'Pneumococcus', 'Polio',
       'Rubella', 'Rotavirus', 'Yellow_fever', 'Diphteria_Tetanus_Pertussis',
       'suicides_rate_100k', 'homicides_rate_100k', 'Population', 'Meningitis',
       'Alzheimer's_disease_and_other_dementias', 'Parkinson's_disease',
       'Nutritional_deficiencies', 'Malaria', 'Drowning',
       'Interpersonal_violence', 'Maternal_disorders', 'HIV/AIDS',
       'Drug_use_disorders', 'Tuberculosis_100k', 'Cardiovascular_diseases',
       'Lower_respiratory_infections', 'Neonatal_disorders',
       'Alcohol_use_disorders', 'Self', 'Exposure_to_forces_of_nature',
       'Diarrheal_diseases', 'Environmental_heat_and_cold_exposure',
       'Neoplasms', 'Conflict_and_terrorism', 'Diabetes_me

## 4.4 Renombrar columnas y ordenarlas

In [122]:
df_analysis_reordered.columns

Index(['Entity', 'Continent', 'Region', 'Code', 'Year', 'Life_expectancy',
       'GDP_capita', 'Share_below_$2.15_a_day', 'Number_below_$2.15_a_day',
       'palma_ratio', 'gini', 'health%_GDP', 'Tuberculosis_vac', 'HepatitisB',
       'Influenza', 'Inactivated_polio', 'Measles', 'Pneumococcus', 'Polio',
       'Rubella', 'Rotavirus', 'Yellow_fever', 'Diphteria_Tetanus_Pertussis',
       'suicides_rate_100k', 'homicides_rate_100k', 'Population', 'Meningitis',
       'Alzheimer's_disease_and_other_dementias', 'Parkinson's_disease',
       'Nutritional_deficiencies', 'Malaria', 'Drowning',
       'Interpersonal_violence', 'Maternal_disorders', 'HIV/AIDS',
       'Drug_use_disorders', 'Tuberculosis_100k', 'Cardiovascular_diseases',
       'Lower_respiratory_infections', 'Neonatal_disorders',
       'Alcohol_use_disorders', 'Self', 'Exposure_to_forces_of_nature',
       'Diarrheal_diseases', 'Environmental_heat_and_cold_exposure',
       'Neoplasms', 'Conflict_and_terrorism', 'Diabetes_me

In [123]:
# Lista de nombres de columnas abreviados con explicaciones
short_columns = [
    'Entity',  # Entidad
    'Continent',  # Continente
    'Region',  # Región
    'Code',  # Código
    'Year',  # Año
    'Population',  # Población
    'Life_expectancy',  # Esperanza de vida
    'GDP_pc',  # PIB per cápita
    'Below_$2.15%',  # % de población por debajo de $2.15 al día
    'Below_$2.15_num',  # Número de personas por debajo de $2.15 al día
    'Palma',  # Ratio de Palma
    'Gini',  # Coeficiente de Gini
    'Health%_GDP',  # % del PIB en sanidad
    'TB_vac',  # Vacuna contra la tuberculosis
    'HepB_vac',  # Vacuna contra la hepatitis B
    'Influenza_vac',  # Vacuna contra la influenza
    'Polio_inact_vac',  # Vacuna de poliomielitis inactivada
    'Measles_vac',  # Vacuna contra el sarampión
    'Pneumo_vac',  # Vacuna contra el neumococo
    'Polio_vac',  # Vacuna contra la poliomielitis
    'Rubella_vac',  # Vacuna contra la rubéola
    'Rota_vac',  # Vacuna contra el rotavirus
    'Yellow_fever_vac',  # Vacuna contra la fiebre amarilla
    'DTP_vac',  # Vacuna contra la difteria, tétanos y tos ferina
    'Suicide',  # Tasa de suicidios
    'Homicide',  # Tasa de homicidios
    'Meningitis',  # Tasa de meningitis
    'Alzheimer',  # Tasa de Alzheimer
    'Parkinson',  # Tasa de Parkinson
    'Nutr_def',  # Deficiencias nutricionales
    'Malaria',  # Tasa de malaria
    'Drowning',  # Tasa de ahogamientos
    'Violence',  # Violencia interpersonal
    'Maternal',  # Desórdenes maternos
    'HIV_AIDS',  # Tasa de VIH/SIDA
    'Drug_use',  # Tasa de uso de drogas
    'TB',  # Tasa de tuberculosis
    'Cardio',  # Enfermedades cardiovasculares
    'Lower_respi',  # Infecciones respiratorias bajas
    'Neonatal',  # Desórdenes neonatales
    'Alcohol',  # Tasa de uso de alcohol
    'Self_harm',  # Autolesiones
    'Nature_forces',  # Exposición a fuerzas de la naturaleza
    'Diarrhea',  # Enfermedades diarreicas
    'Heat_cold',  # Exposición a calor y frío ambientales
    'Neoplasms',  # Neoplasias
    'Conflict_terror',  # Conflictos y terrorismo
    'Diabetes',  # Tasa de diabetes
    'Kidney_dis',  # Enfermedades renales crónicas
    'Poison',  # Envenenamientos
    'Protein_def',  # Deficiencia de proteínas
    'Road_injury',  # Lesiones por accidentes de tráfico
    'Chronic_respi',  # Enfermedades respiratorias crónicas
    'Cirrhosis',  # Cirrosis y otras enfermedades hepáticas crónicas
    'Digestive_dis',  # Enfermedades digestivas
    'Fire_heat',  # Lesiones por fuego, calor y sustancias calientes
    'Acute_hep',  # Hepatitis aguda
    'Country'  # País
]

rename_dict = {
    'Entity': 'Entity',  # Entidad
    'Continent': 'Continent',  # Continente
    'Region': 'Region',  # Región
    'Code': 'Code',  # Código
    'Year': 'Year',  # Año
    'Life_expectancy': 'Life_expectancy',  # Esperanza de vida
    'GDP_capita': 'GDP_pc',  # PIB per cápita
    'Share_below_$2.15_a_day': 'Below_$2.15%',  # % de población por debajo de $2.15 al día
    'Number_below_$2.15_a_day': 'Below_$2.15_num',  # Número de personas por debajo de $2.15 al día
    'palma_ratio': 'Palma',  # Ratio de Palma
    'gini': 'Gini',  # Coeficiente de Gini
    'health%_GDP': 'Health%_GDP',  # % del PIB en sanidad
    'Tuberculosis_vac': 'TB_vac',  # Vacuna contra la tuberculosis
    'HepatitisB': 'HepB_vac',  # Vacuna contra la hepatitis B
    'Influenza': 'Influenza_vac',  # Vacuna contra la influenza
    'Inactivated_polio': 'Polio_inact_vac',  # Vacuna de poliomielitis inactivada
    'Measles': 'Measles_vac',  # Vacuna contra el sarampión
    'Pneumococcus': 'Pneumo_vac',  # Vacuna contra el neumococo
    'Polio': 'Polio_vac',  # Vacuna contra la poliomielitis
    'Rubella': 'Rubella_vac',  # Vacuna contra la rubéola
    'Rotavirus': 'Rota_vac',  # Vacuna contra el rotavirus
    'Yellow_fever': 'Yellow_fever_vac',  # Vacuna contra la fiebre amarilla
    'Diphteria_Tetanus_Pertussis': 'DTP_vac',  # Vacuna contra la difteria, tétanos y tos ferina
    'suicides_rate_100k': 'Suicide',  # Tasa de suicidios
    'homicides_rate_100k': 'Homicide',  # Tasa de homicidios
    'Meningitis': 'Meningitis',  # Tasa de meningitis
    'Alzheimer\'s_disease_and_other_dementias': 'Alzheimer',  # Tasa de Alzheimer
    'Parkinson\'s_disease': 'Parkinson',  # Tasa de Parkinson
    'Nutritional_deficiencies': 'Nutr_def',  # Deficiencias nutricionales
    'Malaria': 'Malaria',  # Tasa de malaria
    'Drowning': 'Drowning',  # Tasa de ahogamientos
    'Interpersonal_violence': 'Violence',  # Violencia interpersonal
    'Maternal_disorders': 'Maternal',  # Desórdenes maternos
    'HIV/AIDS': 'HIV_AIDS',  # Tasa de VIH/SIDA
    'Drug_use_disorders': 'Drug_use',  # Tasa de uso de drogas
    'Tuberculosis_100k': 'TB',  # Tasa de tuberculosis
    'Cardiovascular_diseases': 'Cardio',  # Enfermedades cardiovasculares
    'Lower_respiratory_infections': 'Lower_respi',  # Infecciones respiratorias bajas
    'Neonatal_disorders': 'Neonatal',  # Desórdenes neonatales
    'Alcohol_use_disorders': 'Alcohol',  # Tasa de uso de alcohol
    'Self': 'Self_harm',  # Autolesiones
    'Exposure_to_forces_of_nature': 'Nature_forces',  # Exposición a fuerzas de la naturaleza
    'Diarrheal_diseases': 'Diarrhea',  # Enfermedades diarreicas
    'Environmental_heat_and_cold_exposure': 'Heat_cold',  # Exposición a calor y frío ambientales
    'Neoplasms': 'Neoplasms',  # Neoplasias
    'Conflict_and_terrorism': 'Conflict_terror',  # Conflictos y terrorismo
    'Diabetes_mellitus': 'Diabetes',  # Tasa de diabetes
    'Chronic_kidney_disease': 'Kidney_dis',  # Enfermedades renales crónicas
    'Poisonings': 'Poison',  # Envenenamientos
    'Protein': 'Protein_def',  # Deficiencia de proteínas
    'Road_injuries': 'Road_injury',  # Lesiones por accidentes de tráfico
    'Chronic_respiratory_diseases': 'Chronic_respi',  # Enfermedades respiratorias crónicas
    'Cirrhosis_and_other_chronic_liver_diseases': 'Cirrhosis',  # Cirrosis y otras enfermedades hepáticas crónicas
    'Digestive_diseases': 'Digestive_dis',  # Enfermedades digestivas
    'Fire,_heat,_and_hot_substances': 'Fire_heat',  # Lesiones por fuego, calor y sustancias calientes
    'Acute_hepatitis': 'Acute_hep',  # Hepatitis aguda
    'Country': 'Country'  # País
}

# Aplica el renombrado de columnas
df_analysis_reordered.rename(columns=rename_dict, inplace=True)


In [124]:
# Lista con el nuevo orden de las columnas
ordered_columns = [
        # Información General
        'Continent', 'Region', 'Country','Code', 'Year', 'Population','Entity', 'Life_expectancy',
        # Hipótesis 1: Capacidad económica 
        'GDP_pc', 'Health%_GDP', 'Below_$2.15%', 'Below_$2.15_num', 'Palma', 'Gini',
        # Hipótesis 2: Grado de criminalidad y de factores políticos / geográficos
        'Homicide', 'Conflict_terror', 'Violence', 'Nature_forces', 'Heat_cold',  
        # Hipótesis 3: Desarrollo en Sanidad
        'TB_vac', 'HepB_vac', 'Influenza_vac', 'Polio_inact_vac', 'Measles_vac', 'Pneumo_vac', 'Polio_vac', 
        'Rubella_vac', 'Rota_vac', 'Yellow_fever_vac', 'DTP_vac', 'Diarrhea', 'Kidney_dis',
        'Meningitis', 'Malaria', 'TB', 'Acute_hep','Maternal','Neonatal',
        # Hipótesis 4: Salud física 
        'Cardio', 'HIV_AIDS', 'Neoplasms','Chronic_respi', 'Lower_respi', 'Cirrhosis', 'Digestive_dis',
        'Diabetes','Nutr_def', 'Protein_def',
        # Hipótesis 5: Salud mental
        'Suicide', 'Self_harm', 'Drug_use', 'Alcohol', 'Alzheimer', 'Parkinson',
        # Hipótesis 6: Accidentes
        'Road_injury', 'Fire_heat', 'Drowning', 'Poison'

    ]

# Reordenar las columnas del DataFrame
df_analysis_reordered = df_analysis_reordered[ordered_columns]


# Verificar el nuevo orden de las columnas
df_analysis_reordered.columns


Index(['Continent', 'Region', 'Country', 'Code', 'Year', 'Population',
       'Entity', 'Life_expectancy', 'GDP_pc', 'Health%_GDP', 'Below_$2.15%',
       'Below_$2.15_num', 'Palma', 'Gini', 'Homicide', 'Conflict_terror',
       'Violence', 'Nature_forces', 'Heat_cold', 'TB_vac', 'HepB_vac',
       'Influenza_vac', 'Polio_inact_vac', 'Measles_vac', 'Pneumo_vac',
       'Polio_vac', 'Rubella_vac', 'Rota_vac', 'Yellow_fever_vac', 'DTP_vac',
       'Diarrhea', 'Kidney_dis', 'Meningitis', 'Malaria', 'TB', 'Acute_hep',
       'Maternal', 'Neonatal', 'Cardio', 'HIV_AIDS', 'Neoplasms',
       'Chronic_respi', 'Lower_respi', 'Cirrhosis', 'Digestive_dis',
       'Diabetes', 'Nutr_def', 'Protein_def', 'Suicide', 'Self_harm',
       'Drug_use', 'Alcohol', 'Alzheimer', 'Parkinson', 'Road_injury',
       'Fire_heat', 'Drowning', 'Poison'],
      dtype='object')

In [125]:
df_analysis_reordered.head()

,Continent,Region,Country,Code,Year,Population,Entity,Life_expectancy,GDP_pc,Health%_GDP,...,Suicide,Self_harm,Drug_use,Alcohol,Alzheimer,Parkinson,Road_injury,Fire_heat,Drowning,Poison
0,Europe,Southern Europe,Albania,ALB,2000,3182027,Albania,75.4043,5315.4300,2.765830,...,5.23,4.745403,0.659957,0.439971,13.764811,3.676902,8.139466,0.659957,2.859812,0.754236
1,Europe,Southern Europe,Albania,ALB,2001,3153615,Albania,75.6388,5525.6910,2.753276,...,4.74,4.693027,0.824451,0.475645,14.301048,3.741738,8.149378,0.602483,2.536771,0.729322
2,Europe,Southern Europe,Albania,ALB,2002,3123554,Albania,75.8899,5636.1846,2.588669,...,4.82,4.930281,0.960444,0.480222,15.078977,4.097896,8.579970,0.608281,2.433126,0.704326
3,Europe,Southern Europe,Albania,ALB,2003,3092993,Albania,76.1421,5861.1265,2.670353,...,4.94,5.334639,1.002265,0.517298,15.809929,4.558691,9.279038,0.614292,2.263180,0.646623
4,Europe,Southern Europe,Albania,ALB,2004,3062629,Albania,76.3761,6038.8965,2.820827,...,4.86,5.550787,0.979551,0.522427,16.358495,4.799798,9.468989,0.587730,2.089708,0.587730


In [126]:
list(df_analysis_reordered.columns)

# Encontrar columnas que están en short_columns pero no en ordered_columns
missing_in_ordered = [col for col in short_columns if col not in ordered_columns]

# Encontrar columnas que están en ordered_columns pero no en short_columns
missing_in_short = [col for col in ordered_columns if col not in short_columns]

print("Columnas en short_columns pero no en ordered_columns:", missing_in_ordered)
print("Columnas en ordered_columns pero no en short_columns:", missing_in_short)

Columnas en short_columns pero no en ordered_columns: []
Columnas en ordered_columns pero no en short_columns: []


In [127]:
# Ordenar el DataFrame por las columnas especificadas
df_analysis= df_analysis_reordered.sort_values(by=['Continent', 'Region', 'Country', 'Year'])

## 4.4 Creación de columnas con totales

Finalmente, añadimos columnas con los totales de muertes y los porcentajes por cada una de las hipótesis, que nos servirán para agilizar el análisis posterior

In [128]:
# Crear la columna Total_Hip_2
df_analysis['Hip_2'] = df_analysis[['Homicide', 'Conflict_terror', 'Violence', 'Nature_forces', 'Heat_cold']].sum(axis=1)


# Crear la columna Total_Hip_3
df_analysis['Hip_3'] = df_analysis[['Diarrhea', 'Kidney_dis', 'Meningitis', 'Malaria', 'TB',
                                          'Acute_hep', 'Maternal', 'Neonatal']].sum(axis=1)

# Crear la columna Total_Hip_4
df_analysis['Hip_4'] = df_analysis[['Cardio', 'HIV_AIDS', 'Neoplasms', 'Chronic_respi', 'Lower_respi',
                                          'Cirrhosis', 'Digestive_dis', 'Diabetes', 'Nutr_def', 'Protein_def']].sum(axis=1)

# Crear la columna Total_Hip_5
df_analysis['Hip_5'] = df_analysis[['Suicide', 'Self_harm', 'Drug_use', 'Alcohol', 'Alzheimer', 'Parkinson']].sum(axis=1)

# Crear la columna Total_Hip_6
df_analysis['Hip_6'] = df_analysis[['Road_injury', 'Fire_heat', 'Drowning', 'Poison']].sum(axis=1)

# Crear la columna Total_Vac
df_analysis['Vac_mean'] = df_analysis[['TB_vac', 'HepB_vac', 'Influenza_vac', 'Polio_inact_vac', 'Measles_vac',
                                        'Pneumo_vac', 'Polio_vac', 'Rubella_vac', 'Rota_vac', 'Yellow_fever_vac',
                                        'DTP_vac']].mean(axis=1)

# Crear la columna Total_deaths
df_analysis['Deaths'] = df_analysis[['Hip_2','Hip_3','Hip_4','Hip_5','Hip_6']].sum(axis=1)
# Ver el DataFrame con las nuevas columnas
df_analysis.head()

,Continent,Region,Country,Code,Year,Population,Entity,Life_expectancy,GDP_pc,Health%_GDP,...,Fire_heat,Drowning,Poison,Hip_2,Hip_3,Hip_4,Hip_5,Hip_6,Vac_mean,Deaths
340,Africa,Central Africa,Cameroon,CMR,2000,15091598,Cameroon,52.9317,2673.2998,0.674306,...,2.570967,3.359485,2.007740,7.315329,494.493691,534.701494,34.543176,34.449632,61.5,1105.503322
341,Africa,Central Africa,Cameroon,CMR,2001,15493256,Cameroon,53.1423,2664.7683,0.823361,...,2.555951,3.278846,1.987962,9.191097,500.088555,550.168409,34.957237,34.485972,63.5,1128.891270
342,Africa,Central Africa,Cameroon,CMR,2002,15914029,Cameroon,53.4148,2678.5527,0.951792,...,2.727154,3.625732,2.117628,10.713816,531.600137,574.298313,34.994763,36.382993,66.0,1187.990022
343,Africa,Central Africa,Cameroon,CMR,2003,16354324,Cameroon,53.8826,2767.8074,0.939359,...,2.733222,3.619838,2.121763,10.455949,539.478122,582.622675,35.401580,36.705889,72.0,1204.664216
344,Africa,Central Africa,Cameroon,CMR,2004,16809406,Cameroon,54.1779,2764.6033,0.760645,...,2.706818,3.521838,2.100015,4.170284,539.114827,579.259017,35.450583,36.550964,70.2,1194.545675


In [129]:
# Agrupar por continente y calcular la suma y la media de las columnas
continent_summary = df_analysis.groupby('Continent').agg({
    'Deaths': ['sum', 'mean'],
    'Hip_2': ['sum', 'mean'],
    'Hip_3': ['sum', 'mean'],
    'Hip_4': ['sum', 'mean'],
    'Hip_5': ['sum', 'mean'],
    'Hip_6': ['sum', 'mean'],
    'Vac_mean': 'mean',
    

}).reset_index()

# Renombrar las columnas para mayor claridad
continent_summary.columns = ['_'.join(col).strip() for col in continent_summary.columns.values]

# Ver el DataFrame con estadísticas múltiples por continente
continent_summary

,Continent_,Deaths_sum,Deaths_mean,Hip_2_sum,Hip_2_mean,Hip_3_sum,Hip_3_mean,Hip_4_sum,Hip_4_mean,Hip_5_sum,Hip_5_mean,Hip_6_sum,Hip_6_mean,Vac_mean_mean
0,Africa,509837.004448,944.142601,8946.805234,16.568158,163470.088720,302.722387,302715.902127,560.585004,18233.885042,33.766454,16470.323325,30.500599,79.242787
1,Asia,408329.295949,638.014525,9602.338611,15.003654,50676.402238,79.181878,308355.219750,481.805031,24344.841350,38.038815,15350.493999,23.985147,87.609899
2,Europe,788890.015243,1038.013178,6324.855834,8.322179,18413.022102,24.227661,685004.260871,901.321396,66282.436984,87.213733,12865.439452,16.928210,92.034713
3,North America,160084.057149,615.707912,14873.969713,57.207576,17599.898576,67.691918,111416.847188,428.526335,11082.761888,42.626007,5110.579783,19.656076,88.079476
4,Oceania,26696.295826,667.407396,191.525354,4.788134,1806.028922,45.150723,22000.984921,550.024623,2093.241336,52.331033,604.515294,15.112882,94.603472
5,South America,121733.311529,608.666558,7366.353232,36.831766,12464.794754,62.323974,87821.930609,439.109653,9012.574065,45.062870,5067.658869,25.338294,87.319998


In [130]:
# Calcular los porcentajes directamente en el DataFrame
df_analysis['Hip_2_Percentage'] = (df_analysis['Hip_2'] / df_analysis['Deaths']) * 100
df_analysis['Hip_3_Percentage'] = (df_analysis['Hip_3'] / df_analysis['Deaths']) * 100
df_analysis['Hip_4_Percentage'] = (df_analysis['Hip_4'] / df_analysis['Deaths']) * 100
df_analysis['Hip_5_Percentage'] = (df_analysis['Hip_5'] / df_analysis['Deaths']) * 100
df_analysis['Hip_6_Percentage'] = (df_analysis['Hip_6'] / df_analysis['Deaths']) * 100

# Ver el DataFrame con las nuevas columnas
df_analysis.head()

,Continent,Region,Country,Code,Year,Population,Entity,Life_expectancy,GDP_pc,Health%_GDP,...,Hip_4,Hip_5,Hip_6,Vac_mean,Deaths,Hip_2_Percentage,Hip_3_Percentage,Hip_4_Percentage,Hip_5_Percentage,Hip_6_Percentage
340,Africa,Central Africa,Cameroon,CMR,2000,15091598,Cameroon,52.9317,2673.2998,0.674306,...,534.701494,34.543176,34.449632,61.5,1105.503322,0.661719,44.730186,48.367244,3.124656,3.116194
341,Africa,Central Africa,Cameroon,CMR,2001,15493256,Cameroon,53.1423,2664.7683,0.823361,...,550.168409,34.957237,34.485972,63.5,1128.891270,0.814170,44.299090,48.735288,3.096599,3.054853
342,Africa,Central Africa,Cameroon,CMR,2002,15914029,Cameroon,53.4148,2678.5527,0.951792,...,574.298313,34.994763,36.382993,66.0,1187.990022,0.901844,44.747862,48.342015,2.945712,3.062567
343,Africa,Central Africa,Cameroon,CMR,2003,16354324,Cameroon,53.8826,2767.8074,0.939359,...,582.622675,35.401580,36.705889,72.0,1204.664216,0.867956,44.782448,48.363907,2.938709,3.046981
344,Africa,Central Africa,Cameroon,CMR,2004,16809406,Cameroon,54.1779,2764.6033,0.760645,...,579.259017,35.450583,36.550964,70.2,1194.545675,0.349110,45.131370,48.491994,2.967704,3.059821


In [131]:
# Agrupar por continente y calcular la media de los porcentajes y Vac_mean
continent_summary = df_analysis.groupby('Continent').agg({
    'Hip_2_Percentage': 'mean',
    'Hip_3_Percentage': 'mean',
    'Hip_4_Percentage': 'mean',
    'Hip_5_Percentage': 'mean',
    'Hip_6_Percentage': 'mean',
    'Vac_mean': 'mean'
}).reset_index()

# Ver el DataFrame con los porcentajes y la media por continente
continent_summary

,Continent,Hip_2_Percentage,Hip_3_Percentage,Hip_4_Percentage,Hip_5_Percentage,Hip_6_Percentage,Vac_mean
0,Africa,1.553389,31.615027,59.636010,3.627806,3.567768,79.242787
1,Asia,2.424693,12.830162,74.503579,5.911228,4.330338,87.609899
2,Europe,0.701707,2.473921,86.591896,8.668857,1.563619,92.034713
3,North America,8.467420,11.194964,69.884431,7.046875,3.406310,88.079476
4,Oceania,0.708475,6.609086,82.380930,8.067560,2.233948,94.603472
5,South America,6.497039,10.602936,71.397960,7.158332,4.343733,87.319998


## 4.5 Guardar CSV dataset final

In [132]:
df_analysis.to_csv('df_analysis5_def.csv', index=False)